In [1]:
import tensorflow as tf

/Users/hakan/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np  

In [3]:
m1 = [[1.0, 2.0],
      [3.0, 4.0]]                                  

m2 = np.array([[1.0, 2.0],
               [3.0, 4.0]], dtype=np.float32)      

m3 = tf.constant([[1.0, 2.0],
                  [3.0, 4.0]])                     

print(type(m1))                                    
print(type(m2))                                    
print(type(m3))                                    

t1 = tf.convert_to_tensor(m1, dtype=tf.float32)    
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)    
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)    

print(type(t1))                                    
print(type(t2))                                    
print(type(t3))    

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [5]:
list(t3.op.inputs)

[]

In [6]:
t3.op

<tf.Operation 'Const' type=Const>

In [7]:
t3.op.outputs

[<tf.Tensor 'Const:0' shape=(2, 2) dtype=float32>]

In [8]:
m1 = tf.constant([[1., 2.]])       

m2 = tf.constant([[1],
                  [2]])            

m3 = tf.constant([ [[1,2],
                    [3,4],
                    [5,6]],
                   [[7,8],
                    [9,10],
                    [11,12]]])    

print(m1)                          
print(m2)                          
print(m3)                  

Tensor("Const_3:0", shape=(1, 2), dtype=float32)
Tensor("Const_4:0", shape=(2, 1), dtype=int32)
Tensor("Const_5:0", shape=(2, 3, 2), dtype=int32)


In [10]:
tf.constant(0, shape=[])       

<tf.Tensor 'Const_7:0' shape=() dtype=int32>

In [11]:
tf.ones([500,500]) * 0.5

<tf.Tensor 'mul:0' shape=(500, 500) dtype=float32>

In [12]:
x = tf.constant([[1, 2]])         
negMatrix = tf.negative(x)        
print(negMatrix)     

Tensor("Neg:0", shape=(1, 2), dtype=int32)


Use the TensorFlow operators you’ve learned so far to produce the Gaussian distribution (also known as the normal distribution). See figure 2.3 for a hint. For reference, you can find the probability density of the normal distribution online: https://en.wikipedia.org/wiki/Normal_distribution.

In [ ]:
mean = 0.0
sigma = 1.0
(tf.exp(tf.negative(tf.pow(x - mean, 2.0) / (2.0 * tf.pow(sigma, 2.0) ))) * (1.0 / (sigma * tf.sqrt(2.0 * pi) )))

In [15]:
x = tf.constant([[1., 2.]])           
neg_op = tf.negative(x)               

with tf.Session() as sess:            
    result = sess.run(negMatrix)      
print(result)       

[[-1 -2]]


In [16]:
sess = tf.InteractiveSession()         

x = tf.constant([[1., 2.]])            
negMatrix = tf.negative(x)             

result = negMatrix.eval()              
print(result)                          

sess.close()     

[[-1. -2.]]


In [17]:
x = tf.constant([[1., 2.]])                                                
negMatrix = tf.negative(x)                                                 

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess: 
    result = sess.run(negMatrix)                                           

print(result)                                                              

[[-1. -2.]]


- `Placeholder` —A value that’s unassigned but will be initialized by the session wherever it’s run. Typically, placeholders are the input and output of your model.
- `Variable` —A value that can change, such as parameters of a machine-learning model. Variables must be initialized by the session before they’re used.
- `Constant` —A value that doesn’t change, such as hyperparameters or settings.

https://www.tensorflow.org/api_guides/python/math_ops

In [28]:
sess = tf.InteractiveSession()                    

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]     
spike = tf.Variable(False)                        
spike.initializer.run()                           

for i in range(0, len(raw_data)):                 
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, True)          
        updater.eval()                            
    else:
        tf.assign(spike, False).eval()
    print("Spike", spike.eval())

sess.close()

Spike False
Spike False
Spike True
Spike False
Spike False
Spike True
Spike False
Spike True


In [26]:
sess = tf.InteractiveSession()                                                

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]                                 
spikes = tf.Variable([False] * len(raw_data), name='spikes')                  
saver = tf.train.Saver()

init = tf.global_variables_initializer()
init.run()

for i in range(1, len(raw_data)):                                             
    if raw_data[i] - raw_data[i-1] > 5:                                       
        spikes_val = spikes.eval()                                            
        spikes_val[i] = True                                                  
        updater = tf.assign(spikes, spikes_val)                               
        updater.eval()                                                        

save_path = saver.save(sess, "./spikes.ckpt")                                   
print("spikes data saved in file: %s" % save_path)                            

sess.close()

spikes data saved in file: ./spikes.ckpt


In [27]:
sess = tf.InteractiveSession()

# spikes = tf.Variable([False]*8, name='spikes') # we no longer need it if we run the previous cell on the same notebook!
# spikes.initializer.run()        # You no longer need to initialize this variable because it’ll be directly loaded!!                   
saver = tf.train.Saver()                              

saver.restore(sess, "./spikes.ckpt")                  
print(spikes.eval())                                  

sess.close()

INFO:tensorflow:Restoring parameters from ./spikes.ckpt
[False False  True False False  True False  True]


In [29]:
raw_data = np.random.normal(10, 1, 100)                                    

alpha = tf.constant(0.05)                                                  
curr_value = tf.placeholder(tf.float32)

prev_avg = tf.Variable(0.)

update_avg = alpha * curr_value + (1 - alpha) * prev_avg

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(len(raw_data)):                                         
        curr_avg = sess.run(update_avg, feed_dict={curr_value: raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        print(raw_data[i], curr_avg)

9.966342620078413 0.49831715
10.52455352943744 0.9996289
9.36166852910221 1.4177308
8.600309899939862 1.7768598
10.235372133031515 2.1997855
9.953415168812603 2.5874667
9.385027423525685 2.9273448
10.02673583674066 3.2823143
10.13539452624248 3.6249683
7.647344424615325 3.826087
10.208302637169295 4.145198
9.025571463652925 4.3892164
7.12532482303072 4.5260215
11.146824924907987 4.8570614
7.499086133092989 4.9891624
9.835373689315004 5.231473
9.520278170887508 5.4459133
11.78253783740513 5.7627444
9.872783382257534 5.968246
9.685139294703973 6.154091
10.677697053961092 6.3802714
9.274331451056355 6.5249743
9.429653905699034 6.6702085
10.51499084442454 6.8624477
10.035525015328018 7.0211015
10.013731028370584 7.170733
12.251099967843778 7.4247513
10.596622326030571 7.5833445
11.182600760345629 7.7633076
10.292312340243132 7.8897576
9.181942598456297 7.9543667
8.270323664801623 7.9701643
9.924920905716226 8.067903
7.88104870498648 8.058559
8.371059522959643 8.074184
9.3244348663418 8.136

- Pick out which nodes you care about measuring by annotating them with a summary op.
- Call add_summary on them to queue up data to be written to disk.

In [ ]:
! mkdir logs

In [35]:
! kill $(pgrep -f tensorboard)
# ! rm -rf logs

In [31]:
import os
os.system("tensorboard --logdir=./logs --port=6006 &");

In [34]:
graph = tf.Graph()

with graph.as_default():
    raw_data = np.random.normal(10, 1, 100)

    alpha = tf.constant(0.05, name="alpha")
    curr_value = tf.placeholder(tf.float32, name="curr_value")
    prev_avg = tf.Variable(0., name="prev_avg")

    update_avg = alpha * curr_value + (1 - alpha) * prev_avg

    avg_hist = tf.summary.scalar("running_average", update_avg)           
    value_hist = tf.summary.scalar("incoming_values", curr_value) 
    merged = tf.summary.merge_all()                                       
    writer = tf.summary.FileWriter("./logs")

    init = tf.global_variables_initializer()

with tf.Session(graph=graph) as sess:
    init.run()
    
    for i in range(len(raw_data)):
        summary_str, curr_avg = sess.run([merged, update_avg], feed_dict={curr_value: raw_data[i]})                      
        
        operation = tf.assign(prev_avg, curr_avg)
        operation.eval()
        print(raw_data[i], curr_avg)
        
        writer.add_summary(summary_str, i)       

9.481559047483119 0.47407794
9.04824384740541 0.90278625
9.674803966642784 1.3413872
8.397767572306037 1.6942061
12.236144383119706 2.221303
9.675044007826083 2.59399
10.442957086463696 2.9864385
9.444546013356621 3.3093438
9.752485468613381 3.6315007
10.692277508503615 3.9845395
8.472335898762841 4.208929
10.749797276529199 4.535972
8.505304265525078 4.734439
9.541134068117007 4.9747734
9.39936669720175 5.196003
9.178411258648808 5.395123
10.410278308018318 5.6458807
9.122221353370957 5.8196974
9.560159667329767 6.00672
9.367944427273041 6.1747813
10.199385596338223 6.3760114
10.247314293976718 6.5695767
11.098855473073254 6.7960405
10.661385020330517 6.9893074
9.419209027638162 7.1108027
9.64445966786585 7.2374854
10.037190709689733 7.3774705
9.354826639471188 7.4763384
9.5939239891843 7.5822177
9.733933396264241 7.6898036
9.846569960071243 7.7976418
8.204561696769725 7.817988
10.050930659619274 7.929635
10.874413228180636 8.076874
10.388622106952353 8.192461
9.229635691700164 8.2443

You may need to ensure that the TensorFlow session has ended before starting TensorBoard. If you rerun the cell above, you’ll need to remember to clear the logs directory.